In [1]:
# May 5th, 2020
# APACHE 2.0 License
# Some code modified/reused from https://github.com/sradc/SmallPebble under APACHE 2.0 License terms

In [2]:
__author__ = ["Georgios Kaissis", "Alexander Ziller"]

In [3]:
import sympy as sy
from sympy.abc import *
import numpy as np
from collections import defaultdict
from numbers import Number
from scipy.optimize import shgo
from functools import lru_cache

In [4]:
class Value:
    def __init__(self, value, grads=()):
        self.value = value
        self.grads = grads
    
    def __add__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        return add(self, other)
    
    def __radd__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        return add(other, self)
    
    def __mul__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        return mul(self, other)
    
    def __rmul__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        return mul(other, self)
    
    def __sub__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        return add(self, neg(other))
    
    def __rsub__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        return add(other, neg(self))

    def __truediv__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        return mul(self, inv(other))

    def __rtruediv__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        return mul(other, inv(self))
    
    def __pow__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        return power(self, other)
    
    def __rpow__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        return power(other, self)
    
    def __neg__(self):
        return neg(self)
    
    def exp(self):
        return _exp(self)
    
    def __repr__(self):
        return str(self.value)    
    
def add(a, b):
    value = a.value + b.value    
    grads = (
        (a, 1),
        (b, 1)
    )
    return Value(value, grads)

def mul(a, b):
    value = a.value * b.value
    grads = (
        (a, b.value),
        (b, a.value)
    )
    return Value(value, grads)

def neg(a):
    value = -1 * a.value
    grads = (
        (a, -1),
    )
    return Value(value, grads)

def power(a, b):
    value = a.value**b.value
    grads = (
        (a, b.value*(a.value**(b.value-1))),
        (b, (np.log(a.value) if isinstance(a.value, Number) else sy.log(a.value))*(a.value**b.value))
    )
    return Value(value, grads)

def inv(a):
    value = 1. / a.value
    grads = (
        (a, -1 / a.value**2),
    )
    return Value(value, grads)

def _exp(a):
    value = np.exp(a.value) if isinstance(a.value, Number) else sy.exp(a.value)
    grads = (
        (a, value),
    )
    return Value(value, grads)

    
def log(a):
    value = np.exp(a.value) if isinstance(a.value, Number) else sy.log(a.value)
    grads = (
        (a, 1. / a.value),
    )
    return Value(value, grads)

@lru_cache(maxsize=None)
def grad(Value):
    gradients = defaultdict(lambda: 0)
    def _inner(Value, weight):
        for parent, grad in Value.grads:
            to_par = weight * grad
            gradients[str(parent)] += to_par
            _inner(parent, to_par)
    _inner(Value, weight=1)
    return dict(gradients)

In [5]:
to_values = np.vectorize(lambda x : Value(x))

In [6]:
sigmoid = lambda x: 1/(1+np.exp(-x))

In [113]:
import syft as sy
from syft.lib.adp.entity import Entity
from syft.lib.adp.scalar import Scalar
from syft.lib.adp.adversarial_accountant import AdversarialAccountant
import names
# alice = Scalar(value=0.1, min_val=-1, max_val=1, entity=Entity(name="Alice"))
# bob = Scalar(value=0.01, min_val=-2, max_val=2, entity=Entity(name="Bob"))
# charlie = Scalar(value=0.001, min_val=-2, max_val=2, entity=Entity(name="Charlie"))
# david = Scalar(valuef=0.0001, min_val=-2, max_val=2, entity=Entity(name="David"))

def make_entities(n=100):
    ents = list()
    for i in range(n):
        ents.append(Entity(name=names.get_full_name()))
    return ents

def private(self, min_val, max_val, entities=None):
    
    flat_data = self.flatten()
    if entities is None:
        entities = make_entities(n=len(flat_data))
        
    scalars = list()
    for i in flat_data:
        s = Scalar(value=max(min(float(i), max_val), min_val), min_val=min_val, max_val=max_val, entity=entities[len(scalars)])
        scalars.append(s)
    return to_values(np.array(scalars))
    

x = private(self=np.array([0,1,2,3,4]), min_val=-1, max_val=10)

In [115]:
# x

In [106]:
out = x[1] + x[1]

In [111]:
type(out.value.value)

AttributeError: 'tuple' object has no attribute 'free_symbols'

In [7]:
x = to_values(np.array([i,j,k,l,m,n,o,p])).reshape((8,1)) #8 individuals, 1 feature each
y = to_values(np.array([q,r,s,t,u,v,w,z])).reshape((8,1)) #8 labels, 1 per individual
weights1 = to_values(np.random.uniform(size=(1,12))) #1 feature -> 12 features
weights2 = to_values(np.random.uniform(size=(12,1))) #12 features -> 2 predictions

In [8]:
layer1 = x@weights1
nonlin1 = sigmoid(layer1)
y_pred = nonlin1@weights2
loss = np.mean(np.square(y-y_pred))

In [9]:
%%time
grads = grad(loss)
gradients = {item:grads[item] for item in ("i", "j", "k", "l", "m", "n", "o", "p")}

CPU times: user 5.77 s, sys: 22.5 ms, total: 5.79 s
Wall time: 5.81 s


In [10]:
i_grads = gradients["i"]
i_grads

0.000917289234263041*(-0.089751527921597*q + 0.0644426941140896/(1 + exp(-0.000458644617131521*i)) + 0.0535038305420718/(1 + exp(-0.11511227410626*i)) + 0.0264646714999528/(1 + exp(-0.120036767489687*i)) + 0.0466940644125706/(1 + exp(-0.335310888711682*i)) + 0.0566148092339863/(1 + exp(-0.352757926684868*i)) + 0.0256511451704527/(1 + exp(-0.353901978140667*i)) + 0.016497246454841/(1 + exp(-0.480682924441222*i)) + 0.0405016539183177/(1 + exp(-0.538365557430182*i)) + 0.0592537002226106/(1 + exp(-0.649192077027798*i)) + 0.0766515615055929/(1 + exp(-0.743328555064979*i)) + 0.0445657625775947/(1 + exp(-0.86918849570602*i)) + 0.0750109013333917/(1 + exp(-0.910735853618278*i)))*exp(-0.000458644617131521*i)/(1 + exp(-0.000458644617131521*i))**2 + 0.23022454821252*(-0.0745166012505247*q + 0.0535038305420718/(1 + exp(-0.000458644617131521*i)) + 0.0444217908954375/(1 + exp(-0.11511227410626*i)) + 0.0219724100419862/(1 + exp(-0.120036767489687*i)) + 0.0387679525816804/(1 + exp(-0.335310888711682*i

In [11]:
i_grads.free_symbols

{i, q}

In [12]:
to_optimize = -i_grads #minimize the negative

In [13]:
bounds = [tuple([np.random.uniform(1, 11),np.random.uniform(11,20)]) for _ in range(len(i_grads.free_symbols))]

In [14]:
bounds

[(10.579387090860122, 11.52316946554759),
 (5.872435993196004, 11.020894248689707)]

In [15]:
func = sy.lambdify(list(i_grads.free_symbols), to_optimize)

In [16]:
def func_for_scipy(x):
    return func(*tuple(x))

In [17]:
sol = shgo(func_for_scipy, bounds=bounds)
sol

     fun: 0.03458835150139999
    funl: array([0.03458835])
 message: 'Optimization terminated successfully.'
    nfev: 8
     nit: 2
   nlfev: 3
   nlhev: 0
   nljev: 1
 success: True
       x: array([10.57938709, 11.02089425])
      xl: array([[10.57938709, 11.02089425]])

In [18]:
sol.x

array([10.57938709, 11.02089425])